In [2]:
import os
import numpy as np
import cv2
import pytesseract as pt
import pandas as pd

pt.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [41]:
# Cropping slicers
x_crop = slice(1100,1400)
y_crop = slice(90, 450)


def read_images_from_folder(folder):
    images = []
    filenames = [] 
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            #images.append(pt.image_to_string(img))
            images.append(img)
            filenames.append(filename)
    return images, filenames

def read_img(img):
    img = cv2.imread(img)
    return pt.image_to_string(img)

def parse_substats(string):
    s = pd.Series(string.split('\n'))
    s = s[s.str.count("\+") == 2]
    s = s.str.replace('[^0-9a-zA-Z\+%. ]+', '', regex=True)
    s = s.str.replace('\+ ', '', regex=True).str.split('+')
    s = pd.DataFrame(s.to_list(), columns=['substat','value'])
    s['value'] = s['value'].str.replace('[^0-9.%]','', regex=True)
    return s


def convert_to_bw(image, dark_mode=True):
    # Lower colours are black
    # Higher colours are white

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    if dark_mode:
        light, dark = 1, 0
    else:
        light, dark = 0, 1

    for i in range(gray.shape[0]):
        for j in range(gray.shape[1]):
            if gray[i][j] > 150:
                gray[i][j] = 255 * light
            else:
                gray[i][j] = 255 * dark
                
    return gray        

In [69]:
img = 'img\keqing-flower.png'
original = cv2.imread(img)
crop = original[y_crop, x_crop].copy()
bw_dark = convert_to_bw(crop)
bw_light = convert_to_bw(crop, dark_mode=False)
print(pt.image_to_string(bw_dark))

# cv2.imwrite('dil-fea-bw-dark.png', bw_dark)
# cv2.imwrite('dil-fea-bw-light.png', bw_light)

Gladiator's
Nostalgia .
Flower of Life ,

wo. 1 4, P80.
kkk

+20 -

+ ATK+5.8%

+ CRIT Rate+2.7% ..

- Energy, Recharget16.2% .
+ CRIT DMG+22,5% .

Gladiator's Finale:(2) -
tA

 
 



In [51]:
images, filenames = read_images_from_folder('img')

In [62]:
outputs = pd.DataFrame({
    'original': original_outputs,
    'cropped': cropped_outputs,
    'bw_dark': bw_dark_outputs,
    'bw_light': bw_light_outputs
}, index=names)
outputs['best'] = outputs.idxmax(axis=1)
outputs

,original,cropped,bw_dark,bw_light,best
diluc-circlet.png,3,1,4,4,bw_dark
diluc-feather.png,3,3,1,3,original
diluc-flower.png,4,4,4,3,original
diluc-goblet.png,3,3,4,4,bw_dark
diluc-hourglass.png,3,3,3,3,original
keqing-circlet.png,1,4,3,2,cropped
keqing-feather.png,4,4,4,4,original
keqing-flower.png,0,4,3,4,cropped
keqing-goblet.png,2,2,2,2,original
keqing-hourglass.png,4,4,3,3,original


In [53]:
img = 'img\keqing-goblet.png'

names = []
original_outputs = []
cropped_outputs = []
bw_dark_outputs = []
bw_light_outputs = []

for i, n in zip(images, filenames):
    # original = cv2.imread(i)
    original = i
    crop = original[y_crop, x_crop].copy()
    bw_dark = convert_to_bw(crop, dark_mode=True)
    bw_light = convert_to_bw(crop, dark_mode=False)

    original_str = pt.image_to_string(original)
    crop_str = pt.image_to_string(crop)
    bw_dark_str = pt.image_to_string(bw_dark)
    bw_light_str = pt.image_to_string(bw_light)

    names.append(n)
    original_outputs.append(parse_substats(original_str).shape[0])
    cropped_outputs.append(parse_substats(crop_str).shape[0])
    bw_dark_outputs.append(parse_substats(bw_dark_str).shape[0])
    bw_light_outputs.append(parse_substats(bw_light_str).shape[0])

    # print(f'---------{n}---------')
    # print('\n')
    # print('----ORIGINAL----')
    # # print(original_str)
    # print(parse_substats(original_str))
    # print('\n')
    # print('----CROPPED----')
    # # print(crop_str)
    # print(parse_substats(crop_str))
    # print('\n')
    # print('----BLACK-WHITE----')
    # # print(bw_str)
    # print(parse_substats(bw_str))
    # print('\n\n')
    
